# CLIP Notebook
This is a jupyter notebook file to execute [CLIP](ontolearn.concept_learner.CLIP) and generate predictive results. We recommend you to see the [concept learners](../docs/usage/06_concept_learners.md) guide before continuing with the execution.
Also if you have not done it already, from the main directory "Ontolearn", run the commands for Datasets mentioned [here](https://ontolearn-docs-dice-group.netlify.app/usage/02_installation#download-external-files) to download the datasets.

In [1]:
import json
from ontolearn.knowledge_base import KnowledgeBase
from ontolearn.concept_learner import CLIP
from ontolearn.refinement_operators import ExpressRefinement
from ontolearn.learning_problem import PosNegLPStandard
from owlapy.owl_individual import OWLNamedIndividual, IRI
from ontolearn.utils import setup_logging


/home/nkouagou/.conda/envs/onto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Open `uncle_lp.json` where we have stored the learning problem for the concept of 'Uncle' and the path to the 'family' ontology.

In [2]:
with open('uncle_lp.json') as json_file:
    settings = json.load(json_file)

Create an instance of the class `KnowledeBase` by using the path that is stored in `settings`.

In [3]:
kb = KnowledgeBase(path=settings['data_path'])

Retreive the IRIs of the positive and negative examples of Uncle from `settings` and create an instance of `PosNegLPStandard`. (more info about this [here](../docs/usage/06_concept_learners.md#configure-the-learning-problem))

In [4]:
examples = settings['Uncle']
p = set(examples['positive_examples'])
n = set(examples['negative_examples'])
typed_pos = set(map(OWLNamedIndividual, map(IRI.create, p)))
typed_neg = set(map(OWLNamedIndividual, map(IRI.create, n)))
lp = PosNegLPStandard(pos=typed_pos, neg=typed_neg)

Create a model of [CLIP](ontolearn.concept_learner.CLIP) and fit the learning problem to the model.

In [5]:
op = ExpressRefinement(knowledge_base=kb, use_inverse=False,
                          use_numeric_datatypes=False)

In [6]:
model = CLIP(knowledge_base=kb, path_of_embeddings="../CLIPData/family/embeddings/ConEx_entity_embeddings.csv",
             refinement_operator=op, load_pretrained=True, max_runtime=200)
model.fit(lp)


 Loaded length predictor!

 Loaded length predictor!

 Loaded length predictor!

 Loaded length predictor!


/home/nkouagou/.conda/envs/onto/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0



***** Predicted length: 5 *****

***** Predicted length: 5 *****


In [7]:
list(model.best_descriptions)#[0].quality

[<class 'ontolearn.search.OENode'> at 0x8516977	Brother ⊔ (∃ married.Sister)	Quality:0.93827	Heuristic:0.44752	Depth:2	H_exp:6	|RC|:4	|Indv.|:None,
 <class 'ontolearn.search.OENode'> at 0x8516970	Brother ⊔ (∃ married.Daughter)	Quality:0.92683	Heuristic:0.43264	Depth:2	H_exp:6	|RC|:4	|Indv.|:None,
 <class 'ontolearn.search.OENode'> at 0x8518500	Son ⊔ (∃ married.Sister)	Quality:0.91566	Heuristic:0.42124	Depth:2	H_exp:6	|RC|:4	|Indv.|:None,
 <class 'ontolearn.search.OENode'> at 0x85184f9	Son ⊔ (∃ married.Daughter)	Quality:0.90476	Heuristic:0.40707	Depth:2	H_exp:6	|RC|:4	|Indv.|:None,
 <class 'ontolearn.search.OENode'> at 0x851870d	Male ⊓ (∀ hasParent.Grandparent)	Quality:0.90476	Heuristic:0.40407	Depth:2	H_exp:6	|RC|:7	|Indv.|:None,
 <class 'ontolearn.search.OENode'> at 0x85187ca	Male ⊓ (∀ hasChild.Grandchild)	Quality:0.90476	Heuristic:0.40407	Depth:2	H_exp:6	|RC|:7	|Indv.|:None,
 <class 'ontolearn.search.OENode'> at 0x85184e4	Brother ⊔ (∀ married.Sister)	Quality:0.89412	Heuristic:0.37788

## Retrieve top 3 hypotheses and print them.

In [8]:
hypotheses = list(model.best_hypotheses(n=3))
[print(_) for _ in hypotheses]

OWLObjectUnionOf((OWLClass(IRI('http://www.benchmark.org/family#','Brother')), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLClass(IRI('http://www.benchmark.org/family#','Sister')))))
OWLObjectUnionOf((OWLClass(IRI('http://www.benchmark.org/family#','Brother')), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLClass(IRI('http://www.benchmark.org/family#','Daughter')))))
OWLObjectUnionOf((OWLClass(IRI('http://www.benchmark.org/family#','Son')), OWLObjectSomeValuesFrom(property=OWLObjectProperty(IRI('http://www.benchmark.org/family#','married')),filler=OWLClass(IRI('http://www.benchmark.org/family#','Sister')))))


[None, None, None]